# Statistical Gridding of Data from The Ocean Data Platform

In this notebook we will download ocean temperature measurements from the seas around Scandinavia and create a map of the average summer ocean temperatures, at 10m depth, for the year 2018. This includes:
- Downloading data fron the Ocean Data Platform
- Interpolating each temperature profile to get the temperature at 10m depth
- Create a gridded map of the mean summer temperature using 2D and 3D interpolation techniques

In [ ]:
from odp_sdk import ODPClient

from odp_sdk.utils.numeric import *
from odp_sdk.utils.visual import *

import numpy as np
import pandas as pd
import metpy.interpolate
import metpy
import matplotlib.pyplot as plt

import cmocean #Colormaps for oceanograph

import warnings
warnings.filterwarnings("ignore")

In [ ]:
ODP_API_KEY = "Y2UyNzUzZDctMmZhYi00MGE0LWJkNWItZWIyMmUzMDkwOWQ1"

In [ ]:
client = ODPClient(api_key=ODP_API_KEY)

In [ ]:
df=client.casts(longitude=[-25,35],
                latitude=[50,80],
                timespan=['2018-06-01','2018-08-31'],
                parameters=['date','lon','lat','z','Temperature'],
                include_flagged_data=False)


In [ ]:
df.head()

In [ ]:
df=df[df.z<50] #Only keep data shallower than 50m

## Interpolate casts to desired depths

In [ ]:
df_int=interpolate_casts_to_z(df,'Temperature',[0,10,20]) # This one takes a long time

In [ ]:
df_int.dropna(inplace=True)
df_int=df_int[df_int.Temperature>-2]
df_int.describe()

# Ocean temperature measurements

In [ ]:
df_int=df_int[(df_int.datetime>'2018-06-01')& (df_int.datetime<'2018-08-31') &(df_int.z==10)]

In [ ]:
plot_casts('Temperature',df_int,cmap=cmocean.cm.thermal,vrange=[0,20])
plt.title('Ocean Temperatures at 10m Depth')

# Ocean summer temperature gridding 2D (longitude,latitude)

In [ ]:

points=df_int[['lon','lat']].values.astype('float')
values=df_int['Temperature'].values.astype('float')


int_points=[np.linspace(-25,35,60*10+1),np.linspace(50,80,30*10+1)]


### Mean Summer Temperature - Radial Basis Function (RBF)

In [ ]:
kind='rbf'
grid,g=interpolate_to_grid(points.copy(),values.copy(),int_points.copy(), interp_type=kind,
                               rbf_func='linear',rbf_smooth=0.1,rescale=True)


In [ ]:
plot_grid(grid[0],grid[1],g,cmap=cmocean.cm.thermal,vrange=[0,20],variable_name='Temperature [C]')
plt.title(kind)

### Mean Summer Temperature - Linear + Nearest

In [ ]:
kind='rbf'
grid,g0=interpolate_to_grid(points.copy(),values.copy(),int_points.copy(), interp_type='nearest',rescale=True)
grid,g1=interpolate_to_grid(points.copy(),values.copy(),int_points.copy(), interp_type='linear',rescale=True)

g=g1.copy()
g[np.isnan(g)]=g0[np.isnan(g)] # Filling the empty boundary values in linear interpolation with nearest

In [ ]:
plot_grid(grid[0],grid[1],g,cmap=cmocean.cm.thermal,vrange=[0,20],variable_name='Temperature [C]')
plt.title('Linear + Nearest')

# Ocean temperature gridding 3D (longitude,latitude and time)

In [ ]:
    
df_int['unixtime']=df_int['datetime'].apply(lambda x : x.value) 
df_int.dropna(inplace=True)
points=df_int[['lon','lat','unixtime']].values.astype('float')
values=df_int['Temperature'].values .astype('float')

int_points=[np.linspace(-25,35,60*4+1),
            np.linspace(50,80,30*4+1),
            np.linspace(pd.Timestamp('2018-06-01').value,pd.Timestamp('2018-08-31').value,8)]
    
    

### Mean Summer Temperature - Linear Interpolation 

In [ ]:
grid,g0=interpolate_to_grid(points.copy(),values.copy(),int_points.copy(), interp_type='nearest',rescale=True)
grid,g1=interpolate_to_grid(points.copy(),values.copy(),int_points.copy(), interp_type='linear',rescale=True)
g=g1.copy()
g[np.isnan(g)]=g0[np.isnan(g)] # Filling the empty boundary values in linear interpolation with nearest

In [ ]:
t_mean=np.nanmean(g,axis=2)
plot_grid(grid[0][:,:,0],grid[1][:,:,0],t_mean,cmap=cmocean.cm.thermal,vrange=[0,20],variable_name='Temperature [C]')
plt.title('Linear + nearest')

### Mean Summer Temperature - Radial Basis Function (RBF) 

In [ ]:
grid,g=interpolate_to_grid(points.copy(),values.copy(),int_points.copy(), interp_type='rbf',rbf_smooth=1e-9,rescale=True)

t_mean=np.nanmean(g[:,:],axis=2)
plot_grid(grid[0][:,:,0],grid[1][:,:,0],t_mean,cmap=cmocean.cm.thermal,vrange=[0,20],variable_name='Temperature [C]')#;plt.title(interp_type)


### Standard Deviation Summer Temperature - Radial Basis Function (RBF) 

In [ ]:
t_std=np.nanstd(g[:,:],axis=2)
plot_grid(grid[0][:,:,0],grid[1][:,:,0],t_std,cmap=cmocean.cm.thermal,vrange=[t_std.min(),t_std.max()],variable_name='Temperature [C]')#;plt.title(interp_type)
